### Objective:
Create a code that calculates for a Hessenberg matrix that is similar to an input matrix A using householder reflections.
Reference: https://github.com/zeryabmoussaoui/SVD-Golub-Kahan/blob/master/GOLUB-KAHAN-1965.pdf

In [1]:
import numpy as np
from numpy.linalg import norm

In [2]:
def diagonalize(matrix):
    row, col = np.shape(matrix)
    
    # print(f'Original Matrix:\n{matrix}\n')
    
    if row >= col:
        for i in range(col):
            
            y = matrix[i:row,i]
            norm_y = norm(y)
            vkk = y[0] + norm_y*np.sign(y[0])
            
            h = norm_y * (norm_y + abs(y[0]))
            v = np.concatenate([np.zeros(i), [vkk], matrix[i+1:row,i]])
            L = np.eye(row) - np.outer(v,v) / h

            matrix = L @ matrix
            matrix[abs(matrix) < 1e-10] = 0
            
            if i == col-1:
                break
                
            x = matrix[i,i+1:col]
            norm_x = norm(x)
            vkk = x[0] + norm_x*np.sign(x[0])
            
            h = norm_x * (norm_x + abs(x[0]))
            v = np.concatenate([np.zeros(i+1), [vkk], matrix[i,i+2:col]])
            R = np.eye(col) - np.outer(v,v) / h
            
            matrix = matrix @ R
            matrix[abs(matrix) < 1e-10] = 0
            
            B = matrix[:col,:col]
            dim = col
            
    else:
        for i in range(row):
            
            x = matrix[i,i:col]
            norm_x = norm(x)
            vkk = x[0] + norm_x*np.sign(x[0])
            
            h = norm_x * (norm_x + abs(x[0]))
            v = np.concatenate([np.zeros(i), [vkk], matrix[i,i+1:col]])
            R = np.eye(col) - np.outer(v,v) / h
            
            matrix = matrix @ R
            matrix[abs(matrix) < 1e-10] = 0       
            
            if i == row-1:
                break
            
            y = matrix[i+1:row,i]
            norm_y = norm(y)
            vkk = y[0] + norm_y*np.sign(y[0])
            
            h = norm_y * (norm_y + abs(y[0]))
            v = np.concatenate([np.zeros(i+1), [vkk], matrix[i+2:row,i]])
            L = np.eye(row) - np.outer(v,v) / h
            
            matrix = L @ matrix
            matrix[abs(matrix) < 1e-10] = 0
            
            B = matrix[:row,:row]
            dim = row            
    
    # print(f'B:\n{np.matrix.round(B,2)}\n')
    
    M = np.zeros([dim*2,dim*2])
    M[:dim,-dim:] = B
    M[-dim:,:dim] = B.T
    
    # print(f'Bidiagonal:\n{np.matrix.round(M,2)}\n')

    identity = np.eye(dim*2)
    P = np.zeros([dim*2,dim*2])
    
    if row >= col:
        odd, even = 0, 0
        for i in range(dim*2):
            if (i+1) % 2: 
                m = dim+odd
                odd += 1
            else:
                m = even           
                even += 1
            P[i,:] = identity[m,:]
    else:
        odd, even = 0, 0
        for i in range(dim*2):
            if (i+1) % 2: 
                m = odd
                odd += 1
            else:
                m = dim+even           
                even += 1
            P[i,:] = identity[m,:]        
    
    Tr = P @ M @ P.T
    
    # print(f'Permutaion:\n{P}\n')
    # print(f'Tridiagonal:\n{np.matrix.round(Tr,2)}\n')
    
    return B, M, P, Tr

In [3]:
def singular(matrix):
    
    print(f'A:\n{matrix}\n')
    
    B, M, P, Tr = diagonalize(matrix)
    
    print(f'B:\n{np.matrix.round(B,2)}\n')
    print(f'Bidiagonal:\n{np.matrix.round(M,2)}\n')
    print(f'Permutaion:\n{P}\n')
    print(f'Tridiagonal:\n{np.matrix.round(Tr,2)}\n')
    
    sngb = np.linalg.svd(B)[1]
    sngtr = np.linalg.svd(Tr)[1][1::2]
    sngmx = np.linalg.svd(matrix)[1]
    
    print(f'Singular Values of Bidiagonal: \n{sngb}')
    print(f'Singular Values of Tridiagonal: \n{sngtr}')
    print(f'Singlular Values of Original: \n{sngmx}\n')
    
    mx_b = abs(sngmx - sngb)
    mx_tr = abs(sngmx - sngb)
    b_tr = abs(sngb - sngtr)
    
    print(f'Difference of Singular Values of Orig and Bidiagonal:\n{mx_b}')
    print(f'Difference of Singular Values of Orig and Tridiagonal:\n{mx_tr}')
    print(f'Difference of Singular Values of Bidiagonal and Tridiagonal:\n{b_tr}')

In [4]:
def for_rand_matrix(row, col, iteration = 1000, eps = 1e-10):
    mx_b_similar = 0
    mx_tr_similar = 0
    b_tr_similar = 0

    for i in range(iteration):
        A = np.random.randn(row,col)

        B, M, P, Tr = diagonalize(matrix)

        sngb = np.linalg.svd(B)[1]
        sngtr = np.linalg.svd(Tr)[1][1::2]
        sngmx = np.linalg.svd(matrix)[1]

        mx_b = abs(sngmx - sngb)
        mx_tr = abs(sngmx - sngb)
        b_tr = abs(sngb - sngtr)
        
        mx_b_similar += 1 if max(mx_b) < eps else 0
        mx_tr_similar += 1 if max(mx_tr) < eps else 0
        b_tr_similar += 1 if max(b_tr) < eps else 0

    print(f'For a {row}x{col} matrix:')
    print(f'{mx_b_similar} out of {iteration} iterations: Bidiagonal matrix has similar singular values with Tridiagonal matrix')
    print(f'{mx_tr_similar} out of {iteration} iterations: Tridiagonal matrix has similar singular values with A matrix.')
    print(f'{b_tr_similar} out of {iteration} iterations: Tridiagonal matrix has similar singular values with Bidiagonal matrix.')

**Example 1:** Row > Column

In [5]:
matrix = np.array([[-4, 1,-2],
                   [-5,-9, 2],
                   [ 9, 2,-3],
                   [-7,-5, 8],
                   [ 8,-9,-3]], float)
singular(matrix)

A:
[[-4.  1. -2.]
 [-5. -9.  2.]
 [ 9.  2. -3.]
 [-7. -5.  8.]
 [ 8. -9. -3.]]

B:
[[15.33 -7.25  0.  ]
 [ 0.   -7.52 -7.41]
 [ 0.    0.   -0.09]]

Bidiagonal:
[[ 0.    0.    0.   15.33 -7.25  0.  ]
 [ 0.    0.    0.    0.   -7.52 -7.41]
 [ 0.    0.    0.    0.    0.   -0.09]
 [15.33  0.    0.    0.    0.    0.  ]
 [-7.25 -7.52  0.    0.    0.    0.  ]
 [ 0.   -7.41 -0.09  0.    0.    0.  ]]

Permutaion:
[[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]]

Tridiagonal:
[[ 0.   15.33  0.    0.    0.    0.  ]
 [15.33  0.   -7.25  0.    0.    0.  ]
 [ 0.   -7.25  0.   -7.52  0.    0.  ]
 [ 0.    0.   -7.52  0.   -7.41  0.  ]
 [ 0.    0.    0.   -7.41  0.   -0.09]
 [ 0.    0.    0.    0.   -0.09  0.  ]]

Singular Values of Bidiagonal: 
[17.41059387  9.79588992  0.05966645]
Singular Values of Tridiagonal: 
[17.41059387  9.79588992  0.05966645]
Singlular Values of Original: 
[17.49762154 13.52288804  5.28816976]

Diff

**Example 2:** Column > Row

In [6]:
matrix = matrix.T
singular(matrix)

A:
[[-4. -5.  9. -7.  8.]
 [ 1. -9.  2. -5. -9.]
 [-2.  2. -3.  8. -3.]]

B:
[[15.33  0.    0.  ]
 [-7.25 -7.52  0.  ]
 [ 0.   -7.41 -0.09]]

Bidiagonal:
[[ 0.    0.    0.   15.33  0.    0.  ]
 [ 0.    0.    0.   -7.25 -7.52  0.  ]
 [ 0.    0.    0.    0.   -7.41 -0.09]
 [15.33 -7.25  0.    0.    0.    0.  ]
 [ 0.   -7.52 -7.41  0.    0.    0.  ]
 [ 0.    0.   -0.09  0.    0.    0.  ]]

Permutaion:
[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]

Tridiagonal:
[[ 0.   15.33  0.    0.    0.    0.  ]
 [15.33  0.   -7.25  0.    0.    0.  ]
 [ 0.   -7.25  0.   -7.52  0.    0.  ]
 [ 0.    0.   -7.52  0.   -7.41  0.  ]
 [ 0.    0.    0.   -7.41  0.   -0.09]
 [ 0.    0.    0.    0.   -0.09  0.  ]]

Singular Values of Bidiagonal: 
[17.41059387  9.79588992  0.05966645]
Singular Values of Tridiagonal: 
[17.41059387  9.79588992  0.05966645]
Singlular Values of Original: 
[17.49762154 13.52288804  5.28816976]

Difference

**Example 3:** Column = Row

In [7]:
matrix= np.array([[-24, 41,-21],
                  [-71,-96,  1],
                  [ 92, 12,-93]], float)
singular(matrix)

A:
[[-24.  41. -21.]
 [-71. -96.   1.]
 [ 92.  12. -93.]]

B:
[[118.66 -90.01   0.  ]
 [  0.   -62.11 -20.73]
 [  0.     0.    87.99]]

Bidiagonal:
[[  0.     0.     0.   118.66 -90.01   0.  ]
 [  0.     0.     0.     0.   -62.11 -20.73]
 [  0.     0.     0.     0.     0.    87.99]
 [118.66   0.     0.     0.     0.     0.  ]
 [-90.01 -62.11   0.     0.     0.     0.  ]
 [  0.   -20.73  87.99   0.     0.     0.  ]]

Permutaion:
[[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]]

Tridiagonal:
[[  0.   118.66   0.     0.     0.     0.  ]
 [118.66   0.   -90.01   0.     0.     0.  ]
 [  0.   -90.01   0.   -62.11   0.     0.  ]
 [  0.     0.   -62.11   0.   -20.73   0.  ]
 [  0.     0.     0.   -20.73   0.    87.99]
 [  0.     0.     0.     0.    87.99   0.  ]]

Singular Values of Bidiagonal: 
[154.28088232  90.96824057  46.20810057]
Singular Values of Tridiagonal: 
[154.28088232  90.96824057  46.20810057]
Singlula

**Example 4:** For Loop 4 by 4 Matrix

In [8]:
for_rand_matrix(row = 4, col = 4, iteration = 1000, eps = 1e-10)

For a 4x4 matrix:
1000 out of 1000 iterations: Bidiagonal matrix has similar singular values with Tridiagonal matrix
1000 out of 1000 iterations: Tridiagonal matrix has similar singular values with A matrix.
1000 out of 1000 iterations: Tridiagonal matrix has similar singular values with Bidiagonal matrix.
